In [1]:
import sys
sys.path.append('/home/thiago/Documentos/GitHub/pipeline_de_dados/utils')
from spark_config import SparkConfig
from pyspark.sql import functions as F

In [2]:
spark = SparkConfig()

In [3]:
spark= SparkConfig().spark_config()

24/07/19 11:47:44 WARN Utils: Your hostname, thiago-550XDA resolves to a loopback address: 127.0.1.1; using 192.168.0.62 instead (on interface wlo1)
24/07/19 11:47:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/19 11:47:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.format('parquet').load('s3a://datalake-test-thiago/03-gold/spark/train')

In [5]:
df.show(5)

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
|  1|  Male| 44|              1|       28.0|                 0|  > 2 Years|           Yes|       40454.0|                26.0|    217|       1|
|  2|  Male| 76|              1|        3.0|                 0|   1-2 Year|            No|       33536.0|                26.0|    183|       0|
|  3|  Male| 47|              1|       28.0|                 0|  > 2 Years|           Yes|       38294.0|                26.0|     27|       1|
|  4|  Male| 21|              1|       11.0|                 1|   < 1 Year|            No|       28619.0|               152.0|    203|  

In [6]:
df.dtypes

[('id', 'int'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Driving_License', 'int'),
 ('Region_Code', 'double'),
 ('Previously_Insured', 'int'),
 ('Vehicle_Age', 'string'),
 ('Vehicle_Damage', 'string'),
 ('Annual_Premium', 'double'),
 ('Policy_Sales_Channel', 'double'),
 ('Vintage', 'int'),
 ('Response', 'int')]

In [12]:
df = df.withColumn('Vehicle_Age',
                   F.when(F.col('Vehicle_Age') == '> 2 Years', 'over_2_years')
                    .when(F.col('Vehicle_Age') == '1-2 Year', 'between_1_2_year')
                    .when(F.col('Vehicle_Age') == '< 1 Year', 'below_1_year')
                    .otherwise(F.col('Vehicle_Age')))

df = df.withColumn('Vehicle_Damage',
                   F.when(F.col('Vehicle_Damage') == 'Yes', 1)
                    .when(F.col('Vehicle_Damage') == 'No', 0)
                    .otherwise(F.col('Vehicle_Damage'))
                    .cast('integer'))

In [13]:
df.show(5)

+---+------+---+---------------+-----------+------------------+----------------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|     Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+----------------+--------------+--------------+--------------------+-------+--------+
|  1|  Male| 44|              1|       28.0|                 0|    over_2_years|             1|       40454.0|                26.0|    217|       1|
|  2|  Male| 76|              1|        3.0|                 0|between_1_2_year|             0|       33536.0|                26.0|    183|       0|
|  3|  Male| 47|              1|       28.0|                 0|    over_2_years|             1|       38294.0|                26.0|     27|       1|
|  4|  Male| 21|              1|       11.0|                 1|    below_1_year|             0|       2861